# Indian Car Price Regression Analysis

In [1]:
#importing packages to read in csv files and begin exploratory data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Importing

In [9]:
#reading in data files
df1 = pd.read_csv('Data/CAR DETAILS FROM CAR DEKHO.csv', index_col = 0)
df1.head()

,year,selling_price,km_driven,fuel,seller_type,transmission,owner
name,,,,,,,
Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [10]:
#reading in second data file
df2 = pd.read_csv('Data/car data.csv', index_col = 0)
df2.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
Car_Name,,,,,,,,
ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [51]:
#reading in third data file, we will use this data set for further analysis as it provides more detail than other two.
df = pd.read_csv('Data/Car details v3.csv')
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


## Data Cleaning

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7906 entries, 0 to 8127
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               7906 non-null   object 
 1   year               7906 non-null   int64  
 2   selling_price      7906 non-null   int64  
 3   km_driven          7906 non-null   int64  
 4   fuel               7906 non-null   object 
 5   seller_type        7906 non-null   object 
 6   transmission       7906 non-null   object 
 7   owner              7906 non-null   object 
 8   mileage            7906 non-null   object 
 9   engine             7906 non-null   object 
 10  max_power          7906 non-null   object 
 11  torque             7906 non-null   object 
 12  seats              7906 non-null   float64
 13  selling_price_usd  7906 non-null   float64
 14  mi_driven          7906 non-null   float64
dtypes: float64(3), int64(3), object(9)
memory usage: 988.2+ KB


In [65]:
df.isna().sum()

name                   0
year                   0
selling_price          0
km_driven              0
fuel                   0
seller_type            0
transmission           0
owner                  0
mileage              221
engine               221
max_power            215
torque               222
seats                221
selling_price_usd      0
mi_driven              0
dtype: int64

In [66]:
df.dropna(inplace = True)

In [67]:
df.describe()

,year,selling_price,km_driven,seats,selling_price_usd,mi_driven
count,7906.000000,7.906000e+03,7.906000e+03,7906.000000,7906.000000,7.906000e+03
mean,2013.983936,6.498137e+05,6.918866e+04,5.416393,8447.578371,4.299183e+04
std,3.863695,8.135827e+05,5.679230e+04,0.959208,10576.575729,3.528909e+04
min,1994.000000,2.999900e+04,1.000000e+00,2.000000,389.987000,6.213710e-01
25%,2012.000000,2.700000e+05,3.500000e+04,5.000000,3510.000000,2.174799e+04
50%,2015.000000,4.500000e+05,6.000000e+04,5.000000,5850.000000,3.728226e+04
75%,2017.000000,6.900000e+05,9.542500e+04,5.000000,8970.000000,5.929433e+04
max,2020.000000,1.000000e+07,2.360457e+06,14.000000,130000.000000,1.466720e+06


In [54]:
#prices are listed in Indian Rupee and gas mileage metrics are in kilometers - we will convert to USD and MPG
INR_USD = 0.013 #as of 7/16/21
df['selling_price_usd'] = df['selling_price'] * INR_USD

KM_MI = 0.621371 #converting KM to miles
df['mi_driven'] = df['km_driven'] * KM_MI


In [73]:
#converting mileage to mpg
df['mileage'] = [x[:-5] for x in df['mileage']] #removing text
df['mileage'] = df['mileage'].astype(float) #changing type to floats
df['mileage_mpg'] = df['mileage'] / 0.425143707 #kmpl to mpg conversion 

In [77]:
#converting engine and max power to floats
df['engine'] = [x[:-3] for x in df['engine']] #removing text
df['engine'] = df['engine'].astype(float) #changing type to floats
df['max_power'] = [x[:-4] for x in df['max_power']] #removing text
df['max_power'] = df['max_power'].astype(float) #changing type to floats

In [79]:
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,selling_price_usd,mi_driven,mileage_mpg
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,190Nm@ 2000rpm,5.0,5850.0,90409.4805,55.040213
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,250Nm@ 1500-2500rpm,5.0,4810.0,74564.5200,49.724363
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,"12.7@ 2,700(kgm@ rpm)",5.0,2054.0,86991.9400,41.632981
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,22.4 kgm at 1750-2750rpm,5.0,2925.0,78914.1170,54.099354
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,"11.5@ 4,500(kgm@ rpm)",5.0,1690.0,74564.5200,37.869548


In [82]:
df = df[['name', 'year', 'selling_price_usd', 'mi_driven', 'mileage_mpg', 'fuel', 'seller_type', 'transmission', 'owner', 'engine', 'max_power', 'seats']]

In [85]:
df.head()

,name,year,selling_price_usd,mi_driven,mileage_mpg,fuel,seller_type,transmission,owner,engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,5850.0,90409.4805,55.040213,Diesel,Individual,Manual,First Owner,1248.0,74.00,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,4810.0,74564.5200,49.724363,Diesel,Individual,Manual,Second Owner,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,2054.0,86991.9400,41.632981,Petrol,Individual,Manual,Third Owner,1497.0,78.00,5.0
3,Hyundai i20 Sportz Diesel,2010,2925.0,78914.1170,54.099354,Diesel,Individual,Manual,First Owner,1396.0,90.00,5.0
4,Maruti Swift VXI BSIII,2007,1690.0,74564.5200,37.869548,Petrol,Individual,Manual,First Owner,1298.0,88.20,5.0


## Exploratory Data Analysis